# Livraisons à domicile

Ce fichier contient plusieurs étapes
1. import de librairies essentielles
2. Insertion du fichier csv de clients pour livraison
3. Détermination de la position gps (geocoding)
3. Traitement du fichier et des doublons
4. Mise en place sur une carte

### Pour déterminer l'ordre des livraisons il faut ajouter l'adresse ci-dessous de la livraison

#### Il est important de laisser les guillements simples '' de part et d'autre des noms
-----

In [1]:
Nom_Magasin = 'MAGASIN'
Prenom_Magasin = 'QUIVABIEN'
Adresse_Magasin = '71, en Fournirue'
CP_Magasin = 57000
Ville_Magasin = 'Metz'

---

Il faut ensuite modifier si besoin le nom du fichier .csv 
Pour le moment le fichier est "test.csv" qui 

In [2]:
fichier = 'test.csv'

---

In [3]:
import pylab
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim, GoogleV3, IGNFrance, BANFrance
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium import plugins, IFrame
from folium.features import *

import random
import csv
import simplekml

from pyroutelib3 import Router
import numpy as np

from scipy.spatial import distance_matrix
from scipy.spatial.distance import cdist
from scipy.sparse.csgraph import shortest_path

from tsp_solver.greedy import solve_tsp

pylab.rcParams['figure.figsize'] = (24.0, 12.0) #configure the figure

def lon_lat_to_xy(lon, lat, R = 6371000.0):
    """
    calculates lon, lat coordinates of a point on a sphere with
    radius R
    """
    lon_r = np.radians(lon)
    lat_r = np.radians(lat)

    x =  R * np.cos(lat_r[0]) * lon_r
    y = R * lat_r
    return x,y

def get_distance(routeLatLons):
    
    data = []
    for iter, point in enumerate(routeLatLons):
        data.append([point[0], point[1]])
    
    columns = ['Latitude', 'Longitude']
    temp = pd.DataFrame(data, columns=columns)
    
    x_temp,y_temp = lon_lat_to_xy(temp['Longitude'], temp['Latitude'], R = 6371000.0)
    x = x_temp - x_temp[0]
    y = y_temp - y_temp[0]
    d_x = np.diff(x)
    d_y = np.diff(y)
    ds = np.sqrt(d_x*d_x + d_y*d_y) 
    return np.sum(ds)

On effectue le geocoding dans la cellule suivante

In [4]:
pt_depart = pd.DataFrame({'Nº' : 0, 'Type':['L'], 'Nom':[Nom_Magasin], 'Prénom':[Prenom_Magasin], 'Adresse':[Adresse_Magasin], 'CP':[CP_Magasin], 'Ville':[Ville_Magasin], "Qté totale":['1000000'], 'Tel' : ['06660'], 'Commentaires' :['']})

#On affiche l'adresse totale
pt_depart['addresse_tot'] = pt_depart['Adresse'].astype(str) + ',' + \
                pt_depart['CP'].astype(str) + ',' + \
                pt_depart['Ville'].astype(str) + ', France'

locator = BANFrance()
geocode = RateLimiter(locator.geocode, min_delay_seconds=0.1)
pt_depart['location'] = pt_depart['addresse_tot'].apply(geocode)
#On rajoute un point avec des valeurs nulles
pt_depart['point'] = pt_depart['location'].apply(lambda loc: tuple(loc.point) if loc else None)

#On sépare ensuite l'objet point en latitude, longitude et altitude
pt_depart[['latitude', 'longitude', 'altitude']] = pd.DataFrame(pt_depart['point'].tolist(), index=pt_depart.index)

In [19]:
df = pd.read_csv(fichier, sep=';')
df.head(5)
df.columns
df.dropna(subset=['Nº'], inplace=True)
df['Type'] = df['Type'].str.lower()
df['Nº'] = df['Nº'].astype('int')
df['Tel'] = df['Tel'].astype('str')
df['Tel'] = df['Tel'].str.replace(' ','')
df['Tel'] = df['Tel'].astype('int')
df['Qté totale'] = df['Qté totale'].astype('int')
df['Commentaires'] = df['Commentaires'].replace(np.nan, '', regex=True)
df['Type'] = df['Type'].apply(lambda x: x[0])

liv = df.loc[df['Type'] == 'l']
liv = liv.copy(deep=True)
try:
    groupes = pd.read_csv("groupes.csv", sep=';')
    liv = liv.merge(groupes, how='left').fillna(0)
except:
    liv['Groupe'] = 1
liv['Groupe'] = liv['Groupe'].astype('int')
liv

,Nº,Type,Commentaires,Prénom,Nom,Tel,Adresse,Ville,CP,Qté totale,Groupe
0,1419,l,code 0849,Martin,Pêcheur,642905254,25 rue Bellevue,Montigny-les-Metz,57950,24,2
1,1420,l,,John,LENNON,639098802,5 rue du Moyen Pont,METZ,57000,12,1
2,1421,l,,Lemmy,KILMEISTER,666097562,"32, rue de Metz",BOUSSE,57310,12,0
3,1422,l,,Johnny,CASH,622134456,"6, rue des trois Évêchés",METZ,57070,12,2
4,1422,l,,La,RACLURE,612844564,4 rue Turgot,METZ,57070,24,2


In [46]:
#On affiche l'adresse totale
liv['addresse_tot'] = liv['Adresse'].astype(str) + ', ' + \
                liv['CP'].astype(str) + ', ' + \
                liv['Ville'].astype(str) + ', France'   

#En utilisant un $locator de geopy, on trouve l'adresse et la position
locator = BANFrance()
geocode = RateLimiter(locator.geocode, min_delay_seconds=0.1)
liv['location'] = liv['addresse_tot'].apply(geocode)
#On rajoute un point avec des valeurs nulles
liv['point'] = liv['location'].apply(lambda loc: tuple(loc.point) if loc else None)

#On sépare ensuite l'objet point en latitude, longitude et altitude
liv[['latitude', 'longitude', 'altitude']] = pd.DataFrame(liv['point'].tolist(), index=liv.index)
liv

,Nº,Type,Commentaires,Prénom,Nom,Tel,Adresse,Ville,CP,Qté totale,Groupe,addresse_tot,location,point,latitude,longitude,altitude,index,addresse_BAN
0,1419,l,code 0849,Martin,Pêcheur,642905254,25 rue Bellevue,Montigny-les-Metz,57950,24,2,"25 rue Bellevue, 57950, Montigny-les-Metz, France","(25 Rue Bellevue 57950 Montigny-lès-Metz, (49....","(49.102401, 6.143209, 0.0)",49.102401,6.143209,0.0,0,25 Rue Bellevue 57950 Montigny-lès-Metz
1,1420,l,,John,LENNON,639098802,5 rue du Moyen Pont,METZ,57000,12,1,"5 rue du Moyen Pont, 57000, METZ, France","(Rue du Moyen Pont 57000 Metz, (49.119108, 6.1...","(49.119108, 6.171286, 0.0)",49.119108,6.171286,0.0,1,Rue du Moyen Pont 57000 Metz
2,1421,l,,Lemmy,KILMEISTER,666097562,"32, rue de Metz",BOUSSE,57310,12,0,"32, rue de Metz, 57310, BOUSSE, France","(32 Rue de Metz 57310 Bousse, (49.267948, 6.20...","(49.267948, 6.201441, 0.0)",49.267948,6.201441,0.0,2,32 Rue de Metz 57310 Bousse
3,1422,l,,Johnny,CASH,622134456,"6, rue des trois Évêchés",METZ,57070,12,2,"6, rue des trois Évêchés, 57070, METZ, France","(6 Rue des Trois Evêchés 57070 Metz, (49.10970...","(49.109708, 6.188734, 0.0)",49.109708,6.188734,0.0,3,6 Rue des Trois Evêchés 57070 Metz
4,1422,l,,La,RACLURE,612844564,4 rue Turgot,METZ,57070,24,2,"4 rue Turgot, 57070, METZ, France","(4 Rue Turgot 57070 Metz, (49.109899, 6.187905))","(49.109899, 6.187905, 0.0)",49.109899,6.187905,0.0,4,4 Rue Turgot 57070 Metz


In [21]:
#On ajoute l'adresse exacte dans un champ 
liv['index'] = np.arange(len(liv))
liv['addresse_BAN'] = liv.location.astype(str)
liv['addresse_BAN'] = liv['addresse_BAN'].str.split(',', n = 1)
liv.addresse_BAN = liv.addresse_BAN.str[0]
#On sépare les commandes
liv.addresse_BAN[liv.addresse_BAN.duplicated()].head()
liv['Prénom'].fillna('', inplace=True)

#dupli = df.duplicated(['addresse_BAN'])
dupli = liv[liv.duplicated(['Nom', 'addresse_BAN'], keep=False)]

df2 = liv.groupby(['Nom','addresse_BAN'], as_index=False)[["Qté totale"]].sum()
df3 = liv.drop_duplicates(subset =['Nom','addresse_BAN'], keep ="first", inplace = False)
mergedDf = df2.merge(df3[['index','Nom','Prénom','Adresse','CP','Ville','addresse_BAN','Tel','Commentaires','latitude', 'longitude','Qté totale','Groupe']], on=['Nom','addresse_BAN','Qté totale'])

mergedDf.sort_values(['index'], ascending=1, inplace=True)
mergedDf.reset_index(drop=True, inplace=True)

print(dupli)

Empty DataFrame
Columns: [Nº, Type, Commentaires, Prénom, Nom, Tel, Adresse, Ville, CP, Qté totale, Groupe, addresse_tot, location, point, latitude, longitude, altitude, index, addresse_BAN]
Index: []


In [24]:
pt_depart['x'], pt_depart['y'] = lon_lat_to_xy(pt_depart['longitude'], pt_depart['latitude'], R = 6371000.0)
mergedDf['x'], mergedDf['y'] = lon_lat_to_xy(mergedDf['longitude'], mergedDf['latitude'], R = 6371000.0)
df_array = mergedDf[['x', 'y']].to_numpy()
dist_map = cdist(df_array, df_array)
pd.DataFrame(dist_map)[0].values
to_drop = np.argwhere(pd.DataFrame(dist_map)[0].values > 1.E5)

#df_lat_nulle = mergedDf[mergedDf['latitude'].isnull()]
df_lat_nulle = mergedDf[mergedDf['latitude'].isna()]
for value in to_drop.flatten():
    value_int = int(value)
    df_lat_nulle = df_lat_nulle.append(mergedDf.iloc[value_int]) 
    #mergedDf.drop([value_int])       

df_lat_nulle.drop_duplicates(subset ='addresse_BAN', keep ="first", inplace = True)

final_client = pd.concat([mergedDf, df_lat_nulle,df_lat_nulle]).drop_duplicates(keep=False)
df_lat_nulle.to_csv('clients_erreur.csv', sep=';')
final_client.to_csv('clients_correct.csv', sep=';')

#final_client['commande'] = final_client[['Qté totale']].apply(lambda x: x%3, axis=1)

In [25]:
final_client

,Nom,addresse_BAN,Qté totale,index,Prénom,Adresse,CP,Ville,Tel,Commentaires,latitude,longitude,Groupe,x,y
0,Pêcheur,25 Rue Bellevue 57950 Montigny-lès-Metz,24,0,Martin,25 rue Bellevue,57950,Montigny-les-Metz,642905254,code 0849,49.102401,6.143209,2,447227.671124,5.459938e+06
1,LENNON,Rue du Moyen Pont 57000 Metz,12,1,John,5 rue du Moyen Pont,57000,METZ,639098802,,49.119108,6.171286,1,449271.686121,5.461796e+06
2,KILMEISTER,32 Rue de Metz 57310 Bousse,12,2,Lemmy,"32, rue de Metz",57310,BOUSSE,666097562,,49.267948,6.201441,0,451466.980213,5.478346e+06
3,CASH,6 Rue des Trois Evêchés 57070 Metz,12,3,Johnny,"6, rue des trois Évêchés",57070,METZ,622134456,,49.109708,6.188734,2,450541.906360,5.460750e+06
4,RACLURE,4 Rue Turgot 57070 Metz,24,4,La,4 rue Turgot,57070,METZ,612844564,,49.109899,6.187905,2,450481.554883,5.460772e+06


On réalise le clustering

In [26]:
tournees=[]
for region, df_region in final_client.groupby('Groupe'):
    
    #df_region.to_csv('clients_tournee_' + str(region) + '.csv', sep=';')    
    #path = solve_tsp(dist_map_gps, endpoints = (0,-1))
    
    df_array = df_region[['x', 'y']].to_numpy()
    #pisciculture debut et fin
    df_array = np.vstack(((pt_depart.iloc[0]['x'], pt_depart.iloc[0]['y']),df_array))
    df_array = np.vstack((df_array, (pt_depart.iloc[0]['x'], pt_depart.iloc[0]['y'])))
    dist_map = cdist(df_array, df_array)
    path = solve_tsp(dist_map, endpoints = (0,-1))  
    path_red = path[1:-1]
    sorterIndex = dict(zip(path_red,range(1,len(path_red)+1)))
    df_region_ordered = df_region.copy(deep=True)    
    df_region_ordered['index'] = np.arange(1,len(path_red)+1)
    df_region_ordered['order'] = df_region_ordered['index'].map(sorterIndex)    
    df_region_ordered.sort_values(['order'], ascending=1, inplace=True)
    df_region_ordered.reset_index(drop=True, inplace=True)            
    df_region_reduced = df_region_ordered[["Nom", "Prénom", "Tel", "Adresse", "CP", "Ville", "Qté totale"]]
    df_region_reduced.to_csv('clients_tournee_' + str(int(region)) + '.csv', sep=';')
    tournees.append(df_region_ordered)

In [27]:
pt_depart['order'] = 0
pt_depart['Groupe'] = 0
final_client = pd.concat([pt_depart, final_client], ignore_index=True, join="inner")

for index, tournee in zip(range(len(tournees)),tournees):
    tournees[index] = pd.concat([pt_depart, tournee], ignore_index=False, join="inner")

In [45]:
folium_map = folium.Map(location=[final_client['latitude'].mean(), 
                                    final_client['longitude'].mean()], 
                                    zoom_start=10, tiles='OpenStreetMap')

folium.plugins.BoatMarker([final_client.iloc[0]['latitude'], final_client.iloc[0]['longitude']]).add_to(folium_map)

Tournee_group=[]

color_icons = ['red', 'blue', 'green', 'orange', 'darkblue', 'darkgreen', 'black', 'lightgreen', 'beige', 'pink', 'lightgray', 'lightred', 'purple', 'darkred', 'darkpurple', 'cadetblue', 'gray', 'lightblue']

for number, tournee in zip(range(len(tournees)),tournees):

    Tournee = FeatureGroup(name='Tournee'+str(int(number)))
    for i in range(1,len(tournee)):

        url='https://www.google.fr/maps/dir//' + \
        str(tournee.iloc[i]['latitude'])+ ',' + str(tournee.iloc[i]['longitude'])
        address = str(tournee.iloc[i]['Adresse']) + ", " + str(tournee.iloc[i]['CP']) + ", " + str(tournee.iloc[i]['Ville'])
        nom_prenom = str(tournee.iloc[i]['Nom']) + '\t' + str(tournee.iloc[i]['Prénom'])
        tel = '0' + str(tournee.iloc[i]['Tel'])
        colis = str(tournee.iloc[i]['Qté totale']) + ' bieres'
        commentaires = str(tournee.iloc[i]['Commentaires'])
        lien = '<a href=' + url + '> Maps </a>'

        popup_name = '<b>' + nom_prenom + '</b>' + '<br>' + str(address) + '<br>'  + tel + '<br>' + colis + '<br>' + commentaires + '<br>' + lien

        popup = folium.Popup(popup_name, max_width=650)

        icon_order = DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html='<div style="font-size: 14pt">' + str(int(tournee.iloc[i]['order'])) + '</div>',
            )

        icon = folium.map.Icon(color=color_icons[number%len(color_icons)])        
        folium.Marker([tournee.iloc[i]['latitude'], tournee.iloc[i]['longitude']], icon=icon_order).add_to(Tournee)
        folium.Marker([tournee.iloc[i]['latitude'], tournee.iloc[i]['longitude']], icon=icon, popup=popup).add_to(Tournee)
        Tournee_group.append(Tournee)
                           
for tournee in Tournee_group:
        folium_map.add_child(tournee)
                           
    #FastMarkerCluster(data=list(zip(df['latitude'].values, df['longitude'].values)), popup=df['addresse_tot']).add_to(folium_map)

    #folium_map.add_child(feature_group)
folium.LayerControl().add_to(folium_map)
folium_map.save("map_tournees.html")
folium_map